In [11]:
!pip install confluent-kafka==1.7.0

In [12]:
from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
import sys
from uuid import uuid4

In [13]:
bootstrap_server = "kafka:9092" # Brokers act as cluster entripoints

In [14]:
conf = {'bootstrap.servers': bootstrap_server}

In [15]:
a = AdminClient(conf)

In [16]:
md = a.list_topics(timeout=10)
print(" {} topics:".format(len(md.topics)))
for t in iter(md.topics.values()):
    if t.error is not None:
        errstr = ": {}".format(t.error)
    else:
        errstr = ""
    print("  \"{}\" with {} partition(s){}".format(t, len(t.partitions), errstr))

 5 topics:
  "TemperatureSensorEvent" with 1 partition(s)
  "_schemas" with 1 partition(s)
  "SmokeSensorEvent" with 1 partition(s)
  "__consumer_offsets" with 50 partition(s)
  "RoboticArm" with 1 partition(s)


In [17]:
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *

import time

topic = "SmokeSensorEvent"

def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [18]:
producer_conf = {
        'bootstrap.servers': bootstrap_server,
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': StringSerializer('utf_8')
}

producer = SerializingProducer(producer_conf)

## run the following cell to demostrate that fire is not detected

In [19]:
import json
from IPython.display import clear_output

while True:
    key = "S1"
    value = {"sensor": "S1","smoke": False,"ts":int(time.time())}
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)
    clear_output(wait=True)


{'sensor': 'S1', 'smoke': False, 'ts': 1634566340}
Produced record to topic SmokeSensorEvent partition [0] @ offset 1102


KeyboardInterrupt: 

to interrupt the execution of the cell, prese the square icon in the bar or choose *interrupt kernel* from the *kernel* dropdown menu

## run the following cell to demostrate to detect fire

In [10]:
while True:
    key = "S1"
    value = {"sensor": "S1","smoke": True,"ts":int(time.time())}
    producer.produce(topic=topic, value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)
    clear_output(wait=True)

{'sensor': 'S1', 'smoke': True, 'ts': 1634562910}
Produced record to topic SmokeSensorEvent partition [0] @ offset 975


KeyboardInterrupt: 